# Crime rates in Victoria - spatial visualisation

<img src="images/Crimestoppers_RGB.png", width=250, height=250, ALIGN="right">
This instructional notebook investigates the reported crime rates in Victoria, Australia. This is by no means production code, it's designed to explain some of the steps involved in analysing and visualising data spatially.

Author: [Paul Geil](http://www.ph.unimelb.edu.au/~geilp/index.html)

Data used here include:

- Crime by location from the [Crime Statistics Agency](https://www.crimestatistics.vic.gov.au/) (CSA)
- Victorian Local Government Areas from [data.gov.au](https://data.gov.au/dataset/vic-local-government-areas-psma-administrative-boundaries)

## Table of contents

- [1. Introduction](#1.-Introduction)


- [2. Local Government Areas](#2.-Victorian-Local-Government-Areas)


- [3. Crime rates](#3.-Crime-rates)
    * [3.1 Just the numbers](#3.1-Just-the-numbers)
    * [3.2 Spatial visualisation](#3.2-Spatial-visualisation)


- [4. Digging deeper into the crime statistics](#4.-Digging-deeper-into-the-crime-statistics)
    * [4.1 Crime rate by division](#4.1-Crime-rate-by-division)
    * [4.2 Crime rate by subdivision](#4.2-Crime-rate-by-subdivision)
    
    
- [5. Final remarks](#5.-Final-remarks)

## 1. Introduction

[[ go back to the top ]](#Table-of-contents)

This notebook uses several Python packages. Some come standard with the [Anaconda Python distribution](http://continuum.io/downloads), some do not. The primary libraries are:

* **[NumPy](http://www.numpy.org/)**: a library which adds support for large, multi-dimensional arrays and matrices, with a large collection of high-level mathematical functions to operate on these arrays.
* **[pandas](https://pandas.pydata.org/)**: a library providing high-performance, easy-to-use data structures and data analysis tools.
* **[json](https://www.json.org/)**: a decoder/encoder for JSON-formatted files. JSON (JavaScript Object Notation) is a lightweight data-interchange format.
* **[folium](https://folium.readthedocs.io/en/latest/)**: an interactive spatial visualisation module using Leaflet maps.
* **[bokeh](https://bokeh.pydata.org/en/latest/)**: an interactive visualisation library that targets modern web browsers for presentation.
* **[matplotlib](https://matplotlib.org/)**: a basic plotting library; most other plotting libraries are built on top of it.
* **[seaborn](https://seaborn.pydata.org/)**: an advanced statistical plotting library.

To make sure you have all of the packages you need, install them with `conda`:

    conda install numpy pandas matplotlib seaborn

Alternatively, if you can install the packages with [pip](https://pip.pypa.io/en/stable/installing/) (a Python package manager):

    pip install numpy pandas matplotlib seaborn

## 2. Victorian Local Government Areas

[[ go back to the top ]](#Table-of-contents)

The statistics explored in this notebook have been aggregated by local government area (LGA). LGA boundary data is available from the Australain Government data repository [here](https://data.gov.au/dataset/vic-local-government-areas-psma-administrative-boundaries).

For the purpose of this notebook we'll use data in the form of a GeoJSON file. [GeoJSON](http://geojson.org/) is a geospatial data interchange format based on JavaScript Object Notation (JSON).

Governmental/professional spatial (Geo)JSON files are typically large due to the large number of vertices (i.e. resolution). It's possible to reduce their resolution (this process is called *simplification*). One example command-line tool for this is [`mapshaper`](https://github.com/mbloch/mapshaper). I've used `mapshaper` to reduce the GeoJSON datafile from 18.8MB to 500kb. The bash script I've used (`geojson_simplify.sh`) can be found in the `data` directory of this repo.

In [ ]:
import json

with open('data/VIC_LGA_simplified.geojson') as f:
    json_data = json.load(f)

A quick way to visualise GeoJSON data is with the python module `folium`. Let's take a look at the Victorian LGA boundaries. This map is intereactive - explore!

In [ ]:
kw_VIC = {'location': [-36.5, 145.], 'zoom_start': 7}

import folium
from folium import GeoJson

m = folium.Map(tiles='cartodbpositron', **kw_VIC)
GeoJson(json_data).add_to(m)
m

While `folium` is an easy-to-use module, it does lack some features and aesthetics. We'll use `bokeh` for the rest of this notebook. In order to do so, we need to do some manual extraction of data from the GeoJSON file, so let's take a closer look at the structure and contents of the file.

The GeoJSON file is a collection of *features*. Each feature in the file contains data for an individual LGA. The first feature contains the following data:

In [ ]:
json_data['features'][0]

*geometry* contains a list of *coordinates* (in the form of a polygon, or linear ring of latitudes and longitudes) representing the LGA boundary. *properties* contains additional data about the LGA (such as names and ids). We'll be using the geometric data, the `vic_lga__3` property (LGA name) and the `lg_ply_pid` property (a unique identifier for each polygon).

Note that some LGAs have multiple, unconnected polygons (e.g. Bass Coast), therefore the mapping from `lg_ply_pid` to `vic_lga__3` is many-to-one. Also, there are some features in the collection that don't have geometric data.

Let's extract the data we need from all features with geometric data, and for later convenience put this data into a `pandas` dataframe.. At the same time, we can write a [python dictionary](https://docs.python.org/2/tutorial/datastructures.html#dictionaries) that links unique polygon identifiers to their LGA name.

In [ ]:
names = []       # Name of LGA polygon belongs to
unique_ids = []  # Unique id of polygon
xs = []          # Longitudes
ys = []          # Latitudes
id_dict = {}
for feat in json_data['features']:
    if feat["geometry"] is not None:
        name = feat["properties"]['vic_lga__3']
        unique_id = feat["properties"]['lg_ply_pid']
        names.append(name)
        unique_ids.append(unique_id)
        xs.append([x[0] for x in feat["geometry"]["coordinates"][0]])
        ys.append([y[1] for y in feat["geometry"]["coordinates"][0]])
        id_dict[unique_id] = name
        
import pandas as pd

data = [('Unique ID', unique_ids), ('LGA', names), ('Longs', xs), ('Lats', ys)]
json_df = pd.DataFrame.from_items(data)

The first 5 rows of data are:

In [ ]:
json_df.head()

We'll revisit this data later when we try to merge crime rate data.

## 3. Crime rates

[[ go back to the top ]](#Table-of-contents)

### 3.1 Just the numbers

[[ go back to the top ]](#Table-of-contents)

Crime by location (LGA) for the year ending June 2017 data is available from the Crime Statistics Agency [here](https://www.crimestatistics.vic.gov.au/crime-statistics/latest-crime-data/download-data-4) as a Comma-Spearated Values (CSV) file.

Let's explore the dataset using `pandas`.

In [ ]:
with open('data/Crime by location_2_LGA - year ending June 2017.csv') as f:
    crime_df = pd.read_csv(f, thousands=',')

The first 5 rows of data are:

In [ ]:
crime_df.head()

The CSA [Explanatory Notes](https://www.crimestatistics.vic.gov.au/about-the-data/explanatory-notes) provide additional information about the data the CSA receives from Victoria Police, how it is processed and how to interpret the summary statistics. From these notes we find that ERP stands for Estimated Resident Population. This can be used to calculate the offence rate per 100000 people in the LGA by

Rate = (Offence count / LGA ERP) * 100000

Doing this we have:

In [ ]:
crime_df['Rate'] = (crime_df['Offence Count'] / crime_df['LGA ERP'] * 100000).round().astype('int')
crime_df.head()

For simplicity, let's tidy things up a little.

The `Victorian ERP` column has only one unique value (it's just the Victorian population). Rather than keep the whole column let's just assign the value to the variable `Victorian_ERP`. We may use it later. We won't be needing the Offence Count, LGA ERP or Victorian ERP columns from hereon so let's drop them. While we're at it, let's shorten the `Local Government Area (LGA)` column title to `LGA`.

In [ ]:
Victorian_ERP = crime_df['Victorian ERP'][0]
crime_df = crime_df.drop(['Offence Count', 'LGA ERP', 'Victorian ERP'], axis=1)
crime_df.rename(columns = {'Local Government Area (LGA)':'LGA'}, inplace=True)
crime_df.head()

Out of interest, let's sort the LGAs with respect to crime rate (descending). The five LGAs with highest crime rates for 2017 are:

In [ ]:
crime_df.sort_values(by='Rate', ascending=False).head()

The five LGAs with lowest crime rates for 2017 are:

In [ ]:
crime_df.sort_values(by='Rate', ascending=False).tail()

We can get a better idea of the distribution in crime rates in the LGAs through a histogram.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

ax = sns.distplot(crime_df['Rate'], bins=20, rug=True, kde=False)
ax.set_title('VIC crime rates (all crime, year ending June 2017)')
ax.set(xlabel='Rate per 100,000 population')
plt.show()

This looks fairly [normal](https://en.wikipedia.org/wiki/Normal_distribution) with the exception of two outliers. More useful information can be taken from a box plot. 

In [ ]:
flierprops = dict(marker='o', markerfacecolor='gray', markersize=6)
ax = sns.boxplot(x=crime_df['Rate'], width=0.5, flierprops=flierprops)
ax.set_title('VIC crime rates (all crime, year ending June 2017)')
ax.set(xlabel='Rate per 100,000 population')
plt.show()

The box shows the quartiles of the dataset while the whiskers extend to show the rest of the distribution, except for points that are determined to be outliers (using a method that is a function of the inter-quartile range).

Which LGAs do these outliers correspond to? They are just the two LGAs with highest rates: Melbourne and La Trobe. It may be interesting to follow up on these LGAs, adding more data (e.g. average unemployment rate) and testing for a correlation.

### 3.2 Spatial visualisation

[[ go back to the top ]](#Table-of-contents)

We should first check if the set of LGAs in the two datafiles (GeoJSON and CSV) match. To do this we can look at their differences.

In [ ]:
print(list(set(json_df['LGA']) - set(crime_df['LGA'])))

We want to be able to check the crime rate for any LGA in Victoria. If there are no crime data available then we should be able to see that is the case (e.g. a [`NaN`](https://docs.scipy.org/doc/numpy-1.13.0/user/misc.html) for the rate value). One way to do this is to merge the crime rates we have calculated into the spatial dataframe. However, the rows in the spatial dataframe corresponding to polygons for LGAs we don't have crime data for will not remain after merging. Therefore, we need to create a separate dataframe for any missing spatial data and append it to the merged dataframe. Below, I've written an ad hoc function (`fill_missing_LGAs`) to just this.

Comparing the sets of LGAs, we see that there are no crime data for LGAs in the GeoJSON file which end with (UNINC).

There is another difference though, that if not treated manually will lead to an omission: COLAC OTWAY. The name of this LGA is hyphenated in the crime data, but not in the GeoJSON file. To fix this, let's rename the LGA name in the crime data to match the json data before merging the dataframes.

In [ ]:
crime_df.replace('COLAC-OTWAY', 'COLAC OTWAY', inplace=True)

The missing rows are:

In [ ]:
print(list(set(json_df['LGA']) - set(crime_df['LGA'])))

In [ ]:
def fill_missing_LGAs(rate_df, spatial_df):
    '''
    Adds missing LGA rows in rate_df.
    '''
    import numpy as np
    spatial_LGA_list = list(set(spatial_df['LGA']) - set(rate_df['LGA']))
    rates = [np.nan for lga in spatial_LGA_list]
    #rates = np.zeros(len(spatial_LGA_list))
    data = [('LGA', spatial_LGA_list), ('Rate', rates)]
    missing_df = pd.DataFrame.from_items(data)
    return rate_df.append(missing_df)

Append missing data and merge...

In [ ]:
new_crime_df = fill_missing_LGAs(crime_df, json_df).reset_index(drop=True)
df_merged = json_df.merge(new_crime_df, how='right', on='LGA')
#df_merged

We can now visualise the crime rate data we calculated above using a [choropleth](https://en.wikipedia.org/wiki/Choropleth_map).

In [ ]:
from bokeh.models import ColumnDataSource, HoverTool, LogTicker, LinearColorMapper, ColorBar
from bokeh.plotting import figure, save, output_notebook, show
from bokeh.palettes import YlOrRd7
output_notebook()

In [ ]:
custom_palette = YlOrRd7[::-1][1:]
color_mapper = LinearColorMapper(palette=custom_palette, low=df_merged['Rate'].min(), high=df_merged['Rate'].max())

source = ColumnDataSource(data=dict(
    x=df_merged['Longs'], y=df_merged['Lats'],
    name=df_merged['LGA'], rate=df_merged['Rate']))

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title="VIC crime rates (all crime, year ending June 2017)", tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    plot_width=850, plot_height=600
)
p.grid.grid_line_color = None
p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.8, line_color='black', line_width=0.3)

color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(),
                     label_standoff=6, border_line_color=None,
                     location=(480,560), orientation='horizontal',
                     height=15, width=300,
                     background_fill_alpha=0, scale_alpha=0.8,
                     title='Rate per 100,000 population'
                     )
p.add_layout(color_bar, 'center')

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("LGA", "@name"),("Rate", "@rate")]

show(p)

As you can see, the high crime rate outliers clearly stand out.

## 4. Digging deeper into the crime statistics

[[ go back to the top ]](#Table-of-contents)

So far we've only looked at the crime rates for all crimes taken together - we've ignored the type of offence because the dataset we've been working with doesn't include that information. We've also looked at only one year (2017).

In order to dig deeper into the crime statistics we need more data. Another CSV datafile available from the CSA is `Crime by location_1_Region_PSA_LGA_OffenceType - year ending June 2017.csv`. Here's what the first few rows look like.

In [ ]:
with open('data/Crime by location_1_Region_PSA_LGA_OffenceType - year ending June 2017.csv') as f:
    crime2_df = pd.read_csv(f, thousands=',')
crime2_df.head()

In [ ]:
print('Dataframe shape = {0}'.format(crime2_df.shape))
print()
print('Unique entries in Jul - Jun reference period column:')
print('{0}'.format(crime2_df['Jul - Jun reference period'].unique()))
print()
print('Unique entries in CSA Offence Division:')
print('{0}'.format(crime2_df['CSA Offence Division'].unique()))
print()
print('Unique entries in CSA Offence Subdivision:')
print('{0}'.format(crime2_df['CSA Offence Subdivision'].unique()))

We can see that this dataframe has:
- 26635 rows and 10 columns 
- historical data over 5 years (despite the file name)
- 6 unique broad offence classifications (CSA Offence Division)
- each CSA Offence Division has its own CSA Offence Subdivisions and CSA Offense Group

Information on the CSA Offence Division can be found [here](https://www.crimestatistics.vic.gov.au/about-the-data/classifications-and-victorian-map-boundaries/offence-classification). We are only interested in the CSA Offence Division at the moment, so for simplicity, let's drop most of the columns we don't need, and rename some others. We also need to change the LGAs to uppercase and the hyphenation for COLAC-OTWAY so as to match the json data.

In [ ]:
Victorian_ERP = crime2_df['Victorian ERP'][0]
crime2_df.drop(['Police Region', 'Police Service Area', 'CSA Offence Group', \
                'Victorian ERP'], axis=1, inplace=True)

crime2_df.rename(columns = {'Jul - Jun reference period':'Year', 'Local Government Area':'LGA', \
                            'CSA Offence Division':'Division', 'CSA Offence Subdivision':'Subdivision', \
                            'Offence Count':'Count'}, inplace=True)
crime2_df['LGA'] = crime2_df['LGA'].str.upper()
crime2_df.replace('COLAC-OTWAY', 'COLAC OTWAY', inplace=True)

Together, this 'cleaned-up' dataframe and the spatial data is our starting point for further analysis.

### 4.1 Crime rate by division

[[ go back to the top ]](#Table-of-contents)

We now need to aggregate the data appropriately so we have the counts for each offence division in each LGA. We can then calculate the LGA Offence Rate for each LGA and each offence division for years 2013-2017.

In [ ]:
def rates_by_LGA(df):
    '''
    Calculates all crime rate by LGA.
    '''
    df2 = df.pivot_table(index=['Year','LGA','LGA ERP'], values='Count', aggfunc=sum)
    df2.reset_index(inplace=True)
    df2['Rate'] = (df2['Count'] / df2['LGA ERP'] * 100000).round().astype('int')
    return df2

def rates_by_LGA_and_div(df, division):
    '''
    Calculates crime rate by LGA and division.
    '''
    df2 = df.pivot_table(index=['Year','LGA','Division','LGA ERP'], values='Count', aggfunc=sum)
    df2.reset_index(inplace=True)
    df2['Rate'] = (df2['Count'] / df2['LGA ERP'] * 100000).round().astype('int')
    df2 = df2[df2['Division']==division]
    return df2

def rates_by_criteria(df, year, division=None, subdivision=None, sort=False):
    '''
    Calculates crime rate by LGA for a specific division.
    '''
    if division == None:
        df2 = rates_by_LGA(df)
        df2 = df2[df2['Year']==year]
        if sort:
            df2 = df2.sort_values(by='Rate', ascending=False)
        return df2.drop(['Year', 'LGA ERP', 'Count'], axis=1)
    else:
        if subdivision == None:
            df2 = rates_by_LGA_and_div(df, division)
            df2 = df2[df2['Year']==year]
            if sort:
                df2 = df2.sort_values(by='Rate', ascending=False)
            return df2.drop(['Year', 'LGA ERP', 'Division', 'Count'], axis=1)
        else:
            df2 = rates_by_LGA_and_div_and_subdiv(df, division, subdivision)
            df2 = df2[df2['Year']==year]
            if sort:
                df2 = df2.sort_values(by='Rate', ascending=False)
            return df2.drop(['Year', 'LGA ERP', 'Division', 'Subdivision', 'Count'], axis=1)

The LGAs with highest Crimes against the person-related crime rates for 2017 are:

In [ ]:
year = 2017
division = 'A Crimes against the person'
df = rates_by_criteria(crime2_df, year, division=division, sort=True)
df.head()

Here's the box plot for this distribtion:

In [ ]:
ax = sns.boxplot(x=df['Rate'], width=0.5, flierprops=flierprops)
ax.set_title('VIC crime rates ({0}, year ending June {1})'.format(division,year))
ax.set(xlabel='Rate per 100,000 population')
plt.show()

Append missing data and merge...

In [ ]:
new_df = fill_missing_LGAs(df, json_df)
df_merged = json_df.merge(new_df, how='right', on='LGA')

And here's the choropleth:

In [ ]:
color_mapper = LinearColorMapper(palette=custom_palette,
                                 low=df_merged['Rate'][df_merged['Rate'] > 0.1].min(),
                                 high=df_merged['Rate'].max())

source = ColumnDataSource(data=dict(
    x=df_merged['Longs'], y=df_merged['Lats'],
    name=df_merged['LGA'], rate=df_merged['Rate']
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title='VIC crime rates ({0}, year ending June {1})'.format(division,year),
    tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    plot_width=850, plot_height=600
)
p.grid.grid_line_color = None
p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.8, line_color='black', line_width=0.3)

color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(),
                     label_standoff=6, border_line_color=None,
                     location=(480,560), orientation='horizontal',
                     height=15, width=300,
                     background_fill_alpha=0, scale_alpha=0.8,
                     title='Rate per 100,000 population'
                     )
p.add_layout(color_bar, 'center')

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("LGA", "@name"),("Rate", "@rate")]

show(p)

### 4.2 Crime rate by subdivision

[[ go back to the top ]](#Table-of-contents)

I've written an ad hoc function (`rates_by_LGA_and_div_and_subdiv`) to calculate crime rate by LGA for a given division and subdivision. It takes in the 'cleaned-up' `pandas` dataframe (`crime2_df` in our case) and returns another.

In [ ]:
def rates_by_LGA_and_div_and_subdiv(df, division, subdivision):
    '''
    Calculates crime rate by LGA, division and subdivision.
    '''
    df2 = df.pivot_table(index=['Year','LGA','Division','Subdivision','LGA ERP'], values='Count', aggfunc=sum)
    df2.reset_index(inplace=True)
    df2['Rate'] = (df2['Count'] / df2['LGA ERP'] * 100000).round().astype('int')
    df2 = df2[(df2['Division']==division) & (df2['Subdivision']==subdivision)]
    return df2

Let's us it to find the 2017 results for homicide and related crime rates.

In [ ]:
year = 2017
division = 'A Crimes against the person'
subdivision = 'A10 Homicide and related offences'
df = rates_by_criteria(crime2_df, year, division=division, subdivision=subdivision, sort=True)
df.head()

In [ ]:
ax = sns.boxplot(x=df['Rate'], width=0.5, flierprops=flierprops)
ax.set_title('VIC crime rates ({0}, year ending June {1})'.format(subdivision,year))
ax.set(xlabel='Rate per 100,000 population')
plt.show()

In [ ]:
new_df = fill_missing_LGAs(df, json_df)
df_merged = json_df.merge(new_df, how='right', on='LGA')

In [ ]:
color_mapper = LinearColorMapper(palette=custom_palette,
                                 low=df_merged['Rate'][df_merged['Rate'] > 0.1].min(),
                                 high=df_merged['Rate'].max())

source = ColumnDataSource(data=dict(
    x=df_merged['Longs'], y=df_merged['Lats'],
    name=df_merged['LGA'], rate=df_merged['Rate']
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title='VIC crime rates ({0}, year ending June {1})'.format(subdivision,year),
    tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    plot_width=850, plot_height=600
)
p.grid.grid_line_color = None
p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.8, line_color='black', line_width=0.3)

color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(),
                     label_standoff=6, border_line_color=None,
                     location=(480,560), orientation='horizontal',
                     height=15, width=300,
                     background_fill_alpha=0, scale_alpha=0.8,
                     title='Rate per 100,000 population'
                     )
p.add_layout(color_bar, 'center')

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("LGA", "@name"),("Rate", "@rate")]

show(p)

## 5. Final remarks

[[ go back to the top ]](#Table-of-contents)

There's clearly many improvements that can be made to the procedures above. As pointed out at the beginning, this is just a walkthrough of the steps involved in analysing and visualising data spatially. The code I've written here is certainly not perfect or optimal... I'm an astrophysicist by trade, not a software engineer (but I'm learning!)

A particularly useful improvement would be to include [widgets](https://bokeh.pydata.org/en/latest/docs/user_guide/interaction/widgets.html) (e.g. pull-down selection menus) to save re-running code. This requires writing more functions and [custom JavaScript callbacks](https://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html#userguide-interaction-jscallbacks) which is beyond the instructional scope of this notebook. This feature (and more) will be included in a another repo in the future.

If you've never created a choropleth before and need to I hope you've found something here of use.